In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import datetime
from hmmlearn import hmm

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
# Input data (two first lines for Corbin and Young)

# input_df = pd.read_csv('D:\\EOD\\EOD_20210908.csv',
input_df = pd.read_csv('~/Desktop/Corbin SBU/AMS 520/Project/BofA Projects Data/EOD_20210908.csv',
                       header = None,
                       names = ['Ticker', # Label columns
                                'Date',
                                'Open',
                                'High',
                                'Low',
                                'Close',
                                'Volume',
                                'Dividend',
                                'Stock_split',
                                'Adj_open',
                                'Adj_high',
                                'Adj_low',
                                'Adj_close',
                                'Adj_volume'])

FileNotFoundError: [Errno 2] File D:\EOD\EOD_20210908.csv does not exist: 'D:\\EOD\\EOD_20210908.csv'

In [ ]:
data = input_df.loc[input_df['Ticker'] == 'SPY'] # Select which index to use for analysis
data.reset_index(inplace=True, drop=True)
data.set_index('Date', inplace=True)

In [ ]:
# Proposed Idea: Create a HMM for the recent Neff days, and for all days after the Neff'th day
# Predict which state we are currently in based on the Neff recent days
# Be fully invested if in a positive market, fully divested in a negative market

Neff = 260 #Length of Lookback
Return = 100*(data['Adj_close'] - data['Adj_close'].shift(1)) / data['Adj_close'].shift(1) #return daily percentage returns.
data['Return'] = Return

In [ ]:
data

In [ ]:
#lookback = neff
def get_hvol_yz(df, lookback=Neff):
    """
    Funtion create a serie of OHLC volatility using length of Lookback and data
    
    Parameters
    ----------
    df : data frame that has Adj_open, Adj_high, Adj_low, Adj_close
    lookback : length of lookback
    
    Return
    ------
    percentage of volatility at each date
    """
    o = df.Adj_open
    h = df.Adj_high
    l = df.Adj_low
    c = df.Adj_close
    
    k = 0.34 / (1.34 + (lookback+1)/(lookback-1))
    cc = np.log(c/c.shift(1))
    ho = np.log(h/o)
    lo = np.log(l/o)
    co = np.log(c/o)
    oc = np.log(o/c.shift(1))
    oc_sq = oc**2
    cc_sq = cc**2
    rs = ho*(ho-co)+lo*(lo-co)
    #close_vol = pd.rolling_sum(cc_sq, window=lookback) * (1.0 / (lookback - 1.0))
    close_vol = cc_sq.rolling(lookback).sum() * (1.0 / (lookback - 1.0))

    #open_vol = pd.rolling_sum(oc_sq, window=lookback) * (1.0 / (lookback - 1.0))
    open_vol = oc_sq.rolling(lookback).sum()  * (1.0 / (lookback - 1.0))
    
    #window_rs = pd.rolling_sum(rs, window=lookback) * (1.0 / (lookback - 1.0))
    window_rs = rs.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    
    result = (open_vol + k * close_vol + (1-k) * window_rs).apply(np.sqrt) * np.sqrt(252)
    result[:lookback-1] = 0.0
    
    return  result * 100

new_vol=get_hvol_yz(data,lookback = 260)
data['Volatility'] = new_vol

In [ ]:
# Sources: stlouisfed.org, yahoo finance
# Helpful guids: https://rdrr.io/github/AndreMikulec/econModel/src/R/StressIndex.R
#~/Desktop/Corbin SBU/AMS 520/Project/Project Code/GitHub/Regime-Detection-HMM/Regression_Variables.xlsx
# Regression to use to predict index
reg_data = pd.read_excel('C:\\Users\\ryans\\Desktop\\AMS\\520\\Regression_Variables.xlsx')

reg_data.rename(columns={'observation_date': 'Date'}, inplace=True)
reg_data.set_index('Date', inplace=True)

In [ ]:
reg_data

In [ ]:
data

In [ ]:
merged_data = data.join(reg_data, how='outer').loc[data.index[0]:data.index[-1],].dropna(subset=['Ticker'])

In [ ]:
merged_data.drop(columns=['Ticker',
                          'Open',
                          'High',
                          'Low',
                          'Close',
                          'Volume',
                          'Dividend',
                          'Stock_split',
                          'Adj_open',
                          'Adj_high',
                          'Adj_low',
                          'Adj_close',
                          'Adj_volume',
                          'Volatility'],
                inplace=True)

# Fill all missing data points with the most recently available value
merged_data.fillna(method = 'ffill', inplace = True)

In [ ]:
merged_data

In [ ]:
from sklearn import datasets
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.model_selection import train_test_split

regression_pred = np.array([])
for i in range(len(data) - 2*Neff):

    #print(i)
    
    na_check = pd.isna(merged_data.iloc[0 + i:Neff + i,1:])
    ind_var_available = np.array([])
    
    for j, k in enumerate(na_check):
        if not any(na_check.loc[:,k]):
            ind_var_available = np.append(ind_var_available, j+1)
            
    ind_var_available = ind_var_available.astype(np.int)
    
    X = merged_data.iloc[0+i:Neff+i,ind_var_available].to_numpy()
    y = merged_data['Return'][i+1:Neff+1+i].to_numpy()
    
    lin_reg = LinearRegression()
    lin_reg.fit(X,y)
    regression_pred = np.append(regression_pred, lin_reg.predict([X[-1]])[0])

In [ ]:
from sklearn import datasets
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.model_selection import train_test_split

cv = 10
regression_pred = np.array([])

for i in range(len(data) - 2*Neff):

    #print(i)
    
    na_check = pd.isna(merged_data.iloc[0 + i:Neff + i,1:])
    ind_var_available = np.array([])
    
    for j, k in enumerate(na_check):
        if not any(na_check.loc[:,k]):
            ind_var_available = np.append(ind_var_available, j+1)
            
    ind_var_available = ind_var_available.astype(np.int)

    X = merged_data.iloc[0+i:Neff+i,ind_var_available].to_numpy()
    y = merged_data['Return'][i+1:Neff+1+i].to_numpy()    

    lassoCV = LassoCV(cv=cv)
    lassoCV.fit(X, y)
#     print(len(lassoCV.coef_))
#     print(lassoCV.predict([X[-1]]))
#     print(y[-1])
    regression_pred = np.append(regression_pred, lassoCV.predict([X[-1]])[0])

In [ ]:
reg_predicted_returns = pd.DataFrame(index=data.index[-len(regression_pred):],
                                     data=regression_pred,
                                     columns=['Regression_prediction'])

In [ ]:
reg_predicted_returns

In [ ]:
#Merge reg_predicted_returns to data
data = data.join(reg_predicted_returns, how='outer')

In [ ]:
# Initialize a HMM
states = 2
max_iterations = 100 # For EM algorithm

current_state = np.array([]) # Initialize an array to track the current regimes

# Exclude first (to calculate trailing volatility) and
# second (to have a full set of observations to fit a HMM) Neff observations
for i in range(Neff+1, len(data) - 2*Neff):

    # Initialize a Gaussian HMM
    model = hmm.GaussianHMM(n_components = states, covariance_type="full", n_iter = max_iterations);
    """
    Representation of a hidden Markov model probability distribution. 
    This class allows for easy evaluation of, sampling from, and maximum-likelihood estimation of the parameters of a HMM
    
    Parameters
    ----------
    n_components: Number of States in the model
    covariance_type: String describing the type of covariance parameters to use.
                     Must be one of ‘spherical’, ‘tied’, ‘diag’, ‘full’. Defaults to ‘diag’.
    n_iter : Number of iterations to perform.
    """
    # Pull Neff observations of Volatility and Returns
    observations = data.iloc[Neff+i:Neff*2+i,:].loc[:,['Volatility', 'Return', 'Regression_prediction']].to_numpy()

    model.fit(observations) # Fit the model to the observations
    """
    Estimate model parameters.
    Parameters
    ----------
    observations : List of array-like observation sequences (shape (n_i, n_features)).
    """
    #print(f'i = {i}') # Print to ensure loop is running

    # Model randomly allocates a '0' or a '1' to a state, so check which state has the higher mean return
    if model.means_[0,1] > model.means_[1,1]:
        positive_state = 0
    else:
        positive_state = 1

    predictions = model.predict(observations) # Predict the state for each observation
    """
    Find most likely state sequence corresponding to obs.

    Parameters
    ----------
    observations : List of n_features-dimensional data points. Each row corresponds to a single data point.
    """
    if positive_state == 1: # If the state with the higer mean return is state 1, do nothing, if not...
        pass
    
    else: # Switch the predicted regimes to ensure state 1 is always the regime with a greater mean return
        zeros = np.where(predictions == 0)
        ones = np.where(predictions == 1)
        predictions[zeros] = 1
        predictions[ones] = 0
        
    # Append the current state to the regime tracker
    current_state = np.append(current_state,predictions[-1])
        
# Switch values from type float to type int for later calculations
current_state = current_state.astype(np.int64)

In [ ]:
# Number of regime change in the data sets
total_regime_switches = sum(np.abs(current_state[1:] - current_state[:-1]))
# how many years exist in the data
years_of_data = len(current_state) / 252 # Approximately 252 trading days in a year
avg_regime_changes_per_year = total_regime_switches / years_of_data

print(f'Total regime switches: {total_regime_switches}')
print(f'Average number of regime switches per year: {avg_regime_changes_per_year:.01f}')

In [ ]:
# Plot regime switches to graphically analyze

dates = [datetime.datetime.strptime(d,"%Y-%m-%d").date() for d in data.index[3*Neff + 1:]]

fig, ax = plt.subplots(figsize=(18,3))

formatter = mdates.DateFormatter("%Y")

ax.xaxis.set_major_formatter(formatter)

fmt_half_year = mdates.MonthLocator(interval=24)
ax.xaxis.set_major_locator(fmt_half_year)

ax.plot(dates, current_state, '.', color = 'royalblue');

In [ ]:
state_0 = np.where(current_state == 0)[0] + 3*Neff + 1
state_1 = np.where(current_state == 1)[0] + 3*Neff + 1

In [ ]:
# Calculate and view statistics of the two states

print(f'Number of occurences of state 0: {len(state_0)}')
print(f'Mean return of state 0: {data.iloc[state_0].Return.mean():.03f}')
print(f'Volatility of state 0: {data.iloc[state_0].Return.std():.03f}')
print('\n')
print(f'Number of occurences of state 1: {len(state_1)}')
print(f'Mean return of state 1: {data.iloc[state_1].Return.mean():.03f}')
print(f'Volatility of state 1: {data.iloc[state_1].Return.std():.03f}')

In [ ]:
# Plot the returns of the data, segmented by hidden state

plot = sns.relplot(x = range(0,len(current_state)),
                   y = "Adj_close",
                   data = data[3*Neff+1:],
                   hue = current_state,
                   linewidth = 0,
                   palette = "Set2",
                   s = 10);

plot.fig.set_size_inches(18,10)

In [ ]:
# Invest in treasury if divested

daily_10_year_treas = reg_data.loc[:,'Var3_10_Yr_Treas']**(1/252) - 1
daily_10_year_treas.fillna(method = 'ffill', inplace = True)

In [ ]:
daily_10_year_treas

In [ ]:
data = data.join(daily_10_year_treas, how='outer').dropna()

In [ ]:
portfolio_returns = current_state * data.Return[Neff+1:] / 100 + (1 - current_state) * data.Var3_10_Yr_Treas[Neff+1:] / 100 + 1

In [ ]:
# Calculate the growth of a theoretical portfolio
# Assume fully invested if state 1, fully divested if state 0

current_portfolio = data.Adj_close[Neff+1] * np.cumprod(portfolio_returns)

In [ ]:
# Plot the return portfolio dynamics

plot = sns.relplot(x = range(0,len(current_state)),
                   y = "Adj_close",
                   data = data[Neff+1:],
                   hue = current_state,
                   linewidth = 0,
                   palette = "Set2",
                   s = 10);

plt.plot(range(0,len(current_state)), current_portfolio, color='royalblue')
plt.legend(['HMM Portfolio','SPY (state 1)', 'SPY (state 0)'])

plot.fig.set_size_inches(14,7)

In [ ]:
current_portfolio_returns = current_portfolio.pct_change(1)
current_portfolio_sharpe = current_portfolio_returns.mean()/current_portfolio_returns.std()
current_portfolio_sharpe

In [ ]:
current_portfolio

In [ ]:
np.savetxt('test_portfolio_500.csv', current_portfolio, delimiter=',')

In [ ]:
# Import previous iterations using different Neff window lengths
test_portfolio_750 = np.loadtxt("test_portfolio_750.csv")
test_portfolio_500 = np.loadtxt("test_portfolio_500.csv")
test_portfolio_250 = np.loadtxt("test_portfolio_250.csv")

fig, ax = plt.subplots(figsize=(18,4))

ax.plot(range(0,len(current_state)+500), data.Adj_close[2*250+1:], color='slategray');
ax.plot(range(1000,len(current_state)+500), test_portfolio_750, color='royalblue');
ax.plot(range(500,len(current_state)+500), test_portfolio_500, color='darkgreen');
ax.plot(range(0,len(current_state)+500), test_portfolio_250, color='darkviolet');

ax.legend(['SPY','HMM (750-day rolling window)', 'HMM (500-day rolling window)', 'HMM (250-day rolling window)']);

In [ ]:
# Empirical results

spy_mean_return = data.Return[2*Neff+1:].mean()
spy_vol = data.Return[2*Neff+1:].std()
spy_sharpe = data.Return[2*Neff+1:].mean() / data.Return[2*Neff+1:].std()

test_portfolio_750_returns = (test_portfolio_750[1:] - test_portfolio_750[:-1]) / test_portfolio_750[:-1]
test_portfolio_750_sharpe = test_portfolio_750_returns.mean()/test_portfolio_750_returns.std()

test_portfolio_500_returns = (test_portfolio_500[1:] - test_portfolio_500[:-1]) / test_portfolio_500[:-1]
test_portfolio_500_sharpe = test_portfolio_500_returns.mean()/test_portfolio_500_returns.std()

test_portfolio_250_returns = (test_portfolio_250[1:] - test_portfolio_250[:-1]) / test_portfolio_250[:-1]
test_portfolio_250_sharpe = test_portfolio_250_returns.mean()/test_portfolio_250_returns.std()

print(f'SPY average return: {spy_mean_return:.03f}')
print(f'SPY volatility: {spy_vol:.03f}')
print(f'SPY Sharpe Ratio: {spy_sharpe:.03f}')
print('\n')
print(f'HMM (750-day rolling window) average return: {data.Return[2*Neff+1:].mean() :.05f}')
print(f'HMM (750-day rolling window) volatility: {data.Return[2*Neff+1:].std():.05f}')
print(f'HMM (750-day rolling window) Sharpe Ratio: {test_portfolio_750_sharpe:.03f}')
print('\n')
print(f'HMM (500-day rolling window) average return: {test_portfolio_500_returns.mean():.05f}')
print(f'HMM (500-day rolling window) volatility: {test_portfolio_500_returns.std():.05f}')
print(f'HMM (500-day rolling window) Sharpe Ratio: {test_portfolio_500_sharpe:.03f}')
print('\n')
print(f'HMM (250-day rolling window) average return: {test_portfolio_250_returns.mean():.05f}')
print(f'HMM (250-day rolling window) volatility: {test_portfolio_250_returns.std():.05f}')
print(f'HMM (250-day rolling window) Sharpe Ratio: {test_portfolio_250_sharpe:.03f}')